In [1]:
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy # 导入扩展类
import os
import sys
import click
WIN = sys.platform.startswith('win')
if WIN: # 如果是 Windows 系统，使用三个斜线
    prefix = 'sqlite:///'
else: # 否则使用四个斜线
    prefix = 'sqlite:////'
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = prefix + os.path.join(app.root_path, 'data.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False # 关闭对模型


db = SQLAlchemy(app) # 初始化扩展，传入程序实例 app

class User(db.Model): # 表名将会是 user（自动生成，小写处理）
    id = db.Column(db.Integer, primary_key=True) # 主键
    name = db.Column(db.String(20)) # 名字
class Movie(db.Model): # 表名将会是 movie
    id = db.Column(db.Integer, primary_key=True) # 主键
    title = db.Column(db.String(60)) # 电影标题
    year = db.Column(db.String(4)) # 电影年份

name = 'Grace Zhang'
movies = [
{'title': 'My Neighbor Totoro', 'year': '1988'},
{'title': 'Dead Poets Society', 'year': '1989'},
{'title': 'A Perfect World', 'year': '1993'},
{'title': 'Leon', 'year': '1994'},
{'title': 'Mahjong', 'year': '1996'},
{'title': 'Swallowtail Butterfly', 'year': '1996'},
{'title': 'King of Comedy', 'year': '1999'},
{'title': 'Devils on the Doorstep', 'year': '1999'},
{'title': 'WALL-E', 'year': '2008'},
{'title': 'The Pork of Music', 'year': '2012'},]
# @app.route('/')
# def index():
    
#     return render_template('index.html', name=name, movies=movies)



# def inject_user(): # 函数名可以随意修改
#     user = User.query.first()
#     return dict(user=user) # 需要返回字典，等同于return {'user': user}


# @app.errorhandler(404) # 传入要处理的错误代码
# def page_not_found(e): # 接受异常对象作为参数
#     user = User.query.first()
#     return render_template('404.html', user=user), 404 # 返回模板和状态码

# @app.context_processor
# @app.cli.command() # 注册为命令
# @click.option('--drop', is_flag=True, help='Create after drop.')
# # 设置选项
# def initdb(drop):
#     """Initialize the database."""
#     if drop: # 判断是否输入了选项
#         db.drop_all()
#     db.create_all()
#     click.echo('Initialized database.') # 输出提示信息

In [6]:
# from app import db
app.app_context().push()
db.create_all()


In [36]:
user = User(name='Grey Li') # 创建一个 User 记录
m1 = Movie(title='Leon', year='1994') # 创建一个 Movie 记录
m2 = Movie(title='Mahjong', year='1996') # 再创建一个 Movie
db.session.add(user) # 把新创建的记录添加到数据库会话
db.session.add(m1)
db.session.add(m2)
db.session.commit() #

In [37]:
 movie = Movie.query.all() 

In [38]:
movie

[<Movie 2>, <Movie 3>, <Movie 4>, <Movie 5>, <Movie 6>, <Movie 7>, <Movie 8>]

In [39]:
import pandas as pd

In [40]:
pd.DataFrame(movie)

,0
0,<Movie 2>
1,<Movie 3>
2,<Movie 4>
3,<Movie 5>
4,<Movie 6>
5,<Movie 7>
6,<Movie 8>


In [41]:
 user= User.query.all() 

In [42]:
Movie.query.count()

7

In [43]:
User.query.count()

4

In [44]:
Movie.query.get(1) 

C:\Users\qianyv\AppData\Local\Temp\ipykernel_5520\3913154469.py:1: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Movie.query.get(1)


In [45]:
user = User.query.first() # 读取用户记录
movies = Movie.query.all() # 读取所有电影记录

In [35]:
movie = Movie.query.get(1)
db.session.delete(movie) # 使用 db.session.delete() 方法删除记录，传入模型实例
db.session.commit() # 提交改动

C:\Users\qianyv\AppData\Local\Temp\ipykernel_5520\2806706420.py:1: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  movie = Movie.query.get(1)


UnmappedInstanceError: Class 'builtins.NoneType' is not mapped

In [46]:
from werkzeug.security import generate_password_hash, check_password_hash

In [49]:
from flask import Flask, render_template, request, url_for, redirect, flash
from flask_sqlalchemy import SQLAlchemy # 导入扩展类
from werkzeug.security import generate_password_hash, check_password_hash
import os
import sys
import click
WIN = sys.platform.startswith('win')
if WIN: # 如果是 Windows 系统，使用三个斜线
    prefix = 'sqlite:///'
else: # 否则使用四个斜线
    prefix = 'sqlite:////'
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = prefix + os.path.join(app.root_path, 'data.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False # 关闭对模型
app.config['SECRET_KEY'] = 'dev' # 等同于 app.secret_key = 'dev'


db = SQLAlchemy(app) # 初始化扩展，传入程序实例 app


class User(db.Model):
    __table_args__ = {'extend_existing': True}
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(20))
    username = db.Column(db.String(20)) # 用户名
    password_hash = db.Column(db.String(128)) # 密码散列值

    def set_password(self, password): # 用来设置密码的方法，接受密码作为参数
        self.password_hash = generate_password_hash(password) #将生成的密码保持到对应字段
        
    def validate_password(self, password): # 用于验证密码的方法，接受密码作为参数
        return check_password_hash(self.password_hash, password)
# 返回布尔值
    
class Movie(db.Model): # 表名将会是 movie
    id = db.Column(db.Integer, primary_key=True) # 主键
    title = db.Column(db.String(60)) # 电影标题
    year = db.Column(db.String(4)) # 电影年份


In [50]:
@app.context_processor
def inject_user(): # 函数名可以随意修改
    user = User.query.first()
    return dict(user=user) # 需要返回字典，等同于return {'user': user}


In [51]:
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST': # 判断是否是 POST 请求
        # 获取表单数据
        title = request.form.get('title') # 传入表单对应输入字段的name 值
        year = request.form.get('year')
        # 验证数据
        if not title or not year or len(year) > 4 or len(title)> 60:
            flash('Invalid input.') # 显示错误提示
            return redirect(url_for('index')) # 重定向回主页
        # 保存表单数据到数据库
        movie = Movie(title=title, year=year) # 创建记录
        db.session.add(movie) # 添加到数据库会话
        db.session.commit() # 提交数据库会话
        flash('Item created.') # 显示成功创建的提示
        return redirect(url_for('index')) # 重定向回主页

    user = User.query.first() # 读取用户记录
    movies = Movie.query.all() # 读取所有电影记录
    return render_template('index.html', movies=movies)
